# mmarco dataset

build dataset for splade training, but most of them can be downloaded from huggingface...

# imports/configs

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [3]:
# path = '/content/gdrive/MyDrive/unicamp/IA368DD/final-project'
# data_path = '/content/gdrive/MyDrive/unicamp/IA368DD/final-project/splade/data'

In [6]:
from pathlib import Path

DATA_DIR = Path().parent.absolute()
DATA_DIR

PosixPath('/Users/tcvieira/GitHub/IA368-DD-012023/final-project/notebooks')

In [8]:
#!mkdir raw_data

In [9]:
!pip install ir_datasets -q

In [10]:
import ir_datasets
from tqdm import tqdm

## full collection

In [11]:
def gen_collection_tsv():
    dataset = ir_datasets.load("mmarco/v2/pt")
    target_path = DATA_DIR.joinpath("raw_data/collection.tsv")
    with open(target_path, "w") as fw:
        for doc in tqdm(
            dataset.docs_iter(), total=dataset.docs_count()
        ):
            line = "\t".join([doc.doc_id, doc.text]) + "\n"
            fw.write(line) 

In [12]:
gen_collection_tsv()

[INFO] If you have a local copy of https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/collections/portuguese_collection.tsv, you can symlink it here to avoid downloading it again: /Users/tcvieira/.ir_datasets/downloads/dde80fa2cc5782cae4c40d1127e51958
[INFO] [starting] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/collections/portuguese_collection.tsv
                                           
                                                                                                                                                          [INFO] [finished] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/collections/portuguese_collection.tsv: [03:40] [3.43GB] [15.6MB/s]
100%|██████████| 8841823/8841823 [04:19<00:00, 34031.84it/s] 


## dev (train) and val (dev) queries

In [13]:
def gen_queries_tsv():
    def write_queries_tsv(output_path, queries_iter, total=None):
        with output_path.open("w") as fw:
            for query in tqdm(queries_iter, total=total):
                line = "\t".join([query.query_id, query.text]) + "\n"
                fw.write(line)

    dev_path = DATA_DIR.joinpath("raw_data/queries.dev.small.tsv")
    dataset = ir_datasets.load("mmarco/v2/pt/dev/small")
    write_queries_tsv(
        dev_path,
        dataset.queries_iter(),
        total=dataset.queries_count(),
    )

    train_path = DATA_DIR.joinpath("raw_data/queries.train.tsv")
    dataset = ir_datasets.load("mmarco/v2/pt/train")
    write_queries_tsv(
        train_path,
        dataset.queries_iter(),
        total=dataset.queries_count(),
    )

In [14]:
gen_queries_tsv()

[INFO] [starting] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/qrels.dev.small.tsv
  0%|          | 0/6980 [00:01<?, ?it/s]
[INFO] [finished] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/qrels.dev.small.tsv: [00:00] [143kB] [413kB/s]
[INFO] [starting] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/dev/portuguese_queries.dev.tsv
                                        
                                                                                                                                                          [INFO] [finished] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/dev/portuguese_queries.dev.tsv: [00:01] [4.97MB] [4.73MB/s]
100%|██████████| 6980/6980 [00:04<00:00, 1732.48it/s]
[INFO] If you have a local copy of https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/train/portuguese_queries.train.tsv, you can symlink it here 

## triplets

In [15]:
dataset = ir_datasets.load("mmarco/v2/pt/train")

In [17]:
dataset.docpairs_count()

39780811

In [19]:
def gen_triplets_dev_tsv(total=100_000_000):
    target_path = DATA_DIR.joinpath(
        "raw_data/triplets.train.small.tsv"
    )
    dataset = ir_datasets.load("mmarco/v2/pt/train")
    query_table = {
        query.query_id: query.text for query in dataset.queries_iter()
    }
    docstore = dataset.docs_store()
    with target_path.open("w") as fw:
        for triple in tqdm(
            dataset.docpairs_iter()[:total], total=total
        ):
            query = query_table[triple.query_id]
            positive = docstore.get(triple.doc_id_a).text
            negative = docstore.get(triple.doc_id_b).text
            line = (
                "\t".join(
                    [
                        query,
                        positive,
                        negative,
                    ]
                )
                + "\n"
            )
            fw.write(line)

In [21]:
#gen_triplets_dev_tsv()

took to long...downloaded from huggingface `triplets.train.small.tsv` truncated at 1M

## qrels

In [23]:
from collections import defaultdict
import json

qrel = defaultdict(dict)

with open("raw_data/qrels.dev.small.tsv", 'r') as file:
  for line in file:
    fields = line.split()
    q_id = fields[0]
    doc_id = fields[2]
    rel = fields[3]
    qrel[q_id][doc_id] = rel

with open('raw_data/dev_qrel.json', 'w') as file:
    json.dump(qrel, file)

## val collection

sample from full collection?

In [ ]:
def gen_val_collection_tsv():
    dataset = ir_datasets.load("mmarco/v2/pt")
    target_path = DATA_DIR.joinpath("raw_data/val_collection.tsv")
    with open(target_path, "w") as fw:
        for doc in tqdm(
            dataset.docs_iter(), total=dataset.docs_count()
        ):
            line = "\t".join([doc.doc_id, doc.text]) + "\n"
            fw.write(line)

In [11]:
# adjusted from https://github.com/argonism/JANCE/blob/master/data/gen_jp_data.py

def gen_top1000_dev():
    top1000_path = DATA_DIR.joinpath("raw_data/top1000.dev")
    dataset = ir_datasets.load("mmarco/v2/pt/dev/small")
    query_table = {
        query.query_id: query.text for query in dataset.queries_iter()
    }
    docstore = dataset.docs_store()
    pt_top1000 = []
    with top1000_path.open() as f:
        for line in tqdm(f):
            qid, pid, _, _ = line.split("\t")
            pt_query = query_table[qid]
            pt_passage = docstore.get(pid).text
            pt_top1000.append((qid, pid, pt_query, pt_passage))

    with top1000_path.open("w") as fw:
        for row in tqdm(pt_top1000):
            line = "\t".join(row) + "\n"
            fw.write(line)


def gen_all():
    gen_queries_tsv()
    gen_collection_tsv()
    gen_triples_small_tsv()
    gen_top1000_dev()

In [ ]:
# gen_all()

[INFO] [starting] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/qrels.dev.small.tsv
  0%|          | 0/6980 [00:00<?, ?it/s]

[INFO] [finished] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/qrels.dev.small.tsv: [00:00] [143kB] [2.25MB/s]
  0%|          | 0/6980 [00:00<?, ?it/s]
https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/qrels.dev.small.tsv: [00:00] [143kB] [1.91MB/s]
[INFO] [starting] https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/dev/portuguese_queries.dev.tsv
  0%|          | 0/6980 [00:01<?, ?it/s]
https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/dev/portuguese_queries.dev.tsv: 0.0%| 0.00/4.97M [00:00<?, ?B/s]
https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/dev/portuguese_queries.dev.tsv: 6.1%| 303k/4.97M [00:00<00:01, 2.94MB/s]
https://huggingface.co/datasets/unicamp-dl/mmarco/resolve/main/data/google/queries/d

KeyboardInterrupt: ignored